# Imports

In [1]:
import pandas as pd
from Models.sql_models import *
from Models.scraping_models import *

# Abrir Conexão SQL

In [2]:
servidor  = '{SQL Server}'
usuario = 'Doug'
database_sql = 'B3'
sql_conn = SQL_Connection(servidor, usuario, database_sql)
sql_conn.connect()
cursor = sql_conn.connection.cursor()

Conexão estabelecida com sucesso.


# Envio dos dados coletados(execuçao diária)

## Fiis de dados

### Coleta de dados

In [3]:
# Local coletado dos FIIs
fiis_data = 'https://www.clubefii.com.br/fundos_imobiliarios_ranking/2024#'

# Formatação dos dados para o SQL 
df = scraping_request(fiis_data)

df[['Valor da Ação', 'Variação', 'Data', 'Hora', 'Resto']] = df['COTAÇÃO ATUAL'].str.split(' ', n=4, expand=True) 

# Colunas indesejadas
cols_to_exclude = ['VARIAÇÃO COTA  EM 2024', 'VARIAÇÃO COTA + PROVENTOS  EM 2024', 'Resto', 'COTAÇÃO ATUAL', 'UPSIDE/DOWNSIDE PARA ATINGIR VALOR PATRIMONIAL', 'FEED', 'P/VPA', 'Hora' , 'YIELD  3 MESES', 'YIELD  6 MESES']
df = df.drop(columns= cols_to_exclude)

# Ordenação desejada
df = df[['CÓDIGO', 'NOME', 'Valor da Ação', 'Variação', 'Data', 'YIELD  1 MES', 'YIELD  12 MESES']]

# Conversão de colunas para Float
cols_to_transform = ['Valor da Ação', 'Variação', 'YIELD  1 MES', 'YIELD  12 MESES']

df = float_transformer(df,cols_to_transform)

# Formatação de colunas
df['DI Mensal'] = (df['Valor da Ação'] * (df['YIELD  1 MES'] / 100)).astype(float)
df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
df['Data'] = df['Data'].dt.strftime('%d/%m/%Y')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CÓDIGO           104 non-null    object 
 1   NOME             104 non-null    object 
 2   Valor da Ação    104 non-null    float64
 3   Variação         104 non-null    float64
 4   Data             104 non-null    object 
 5   YIELD  1 MES     104 non-null    float64
 6   YIELD  12 MESES  104 non-null    float64
 7   DI Mensal        104 non-null    float64
dtypes: float64(5), object(3)
memory usage: 6.6+ KB


### Vizualização do dataframe

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CÓDIGO           104 non-null    object 
 1   NOME             104 non-null    object 
 2   Valor da Ação    104 non-null    float64
 3   Variação         104 non-null    float64
 4   Data             104 non-null    object 
 5   YIELD  1 MES     104 non-null    float64
 6   YIELD  12 MESES  104 non-null    float64
 7   DI Mensal        104 non-null    float64
dtypes: float64(5), object(3)
memory usage: 6.6+ KB


In [6]:
df

,CÓDIGO,NOME,Valor da Ação,Variação,Data,YIELD 1 MES,YIELD 12 MESES,DI Mensal
0,HTMX11,Hotel Maxinvest,176.89,0.50,30/04/2024,1.24,16.02,2.193436
1,HGBS11,Hedge Brasil Shopping,222.69,0.33,30/04/2024,0.79,9.63,1.759251
2,HGPO11,CSHG Prime Offices,306.92,0.58,30/04/2024,0.49,6.00,1.503908
3,RCRB11,Rio Bravo Renda Corporativa,159.19,2.36,30/04/2024,0.58,6.19,0.923302
4,HGLG11,CSHG Logística,166.63,-0.06,30/04/2024,0.66,8.16,1.099758
...,...,...,...,...,...,...,...,...
99,BLMG11,BlueMacaw Logística,38.66,-0.18,30/04/2024,1.14,18.03,0.440724
100,VSLH11,Versalhes Recebíveis Imobiliários,3.50,-0.28,30/04/2024,1.14,12.00,0.039900
101,BROF11,BRPR Corporate Offices,55.53,-2.41,30/04/2024,0.97,12.26,0.538641
102,TORD11,Tordesilhas EI,1.83,0.00,30/04/2024,0.00,0.00,0.000000


### Envio SQL

In [13]:
# Tabela que receberá os dados
table_name = 'FIIS_Consol'

if not SQLHandler(cursor).table_exists(table_name): # Confere se a tabela ja existe para criar ou não.
    # Executar ação se a tabela não existir
    SQLHandler(cursor).sql_format_table(df, table_name)
    SQLHandler(cursor).sql_format_row(df, table_name)
    sql_conn.commit()
else:
    # Executar ação se a tabela existir
    SQLHandler(cursor).sql_format_row(df, table_name)
    sql_conn.commit()

Transação comitada com sucesso.


## Fiis de Classificação

### Coleta de dados

In [ ]:
# Local coletado dos classificadores fees
fiis_classification = 'https://www.clubefii.com.br/fundo_imobiliario_lista#'

# Limpeza e estruturação dos dados (CLASSIFICADOR)
df = scraping_request(fiis_classification)

# Colunas indesejadas
cols_to_exclude = ['VALOR COTA', 'RELATÓRIOS DE ANÁLISE', 'FEED', 'VALOR IPO','DATA IPO']
df = df.drop(columns= cols_to_exclude)

In [ ]:
df.info()

### Vizualização do dataframe

In [ ]:
df

### Envio SQL

In [ ]:
# Enviar a tabela para o SQL
table_name = 'FIIS_Classification'

if not SQLHandler(cursor).table_exists(table_name): # Confere se a tabela ja existe para criar ou não.
    # Executar ação se a tabela não existir
    SQLHandler(cursor).sql_format_table(df, table_name)
    SQLHandler(cursor).sql_format_row(df, table_name)
    sql_conn.commit()
else:
    pass